In [1]:
import pandas as pd
import numpy as np
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
url = 'https://www2.aneel.gov.br/aplicacoes/indicadores_de_qualidade/decFecSegMensal.cfm?mes={}&ano={}}&regiao=SE&distribuidora={}}&tipo=d'

In [3]:

lista_ano = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
lista_distribuidores = [383,382,5274,6585,6612]
lista_mes = [1,2,3,4,5,6,7,8,9,10,11,12]

In [4]:

for ano in lista_ano:
    for distribuidora in lista_distribuidores:
        for mes in lista_mes:
            scraping = pd.read_html('https://www2.aneel.gov.br/aplicacoes/indicadores_de_qualidade/decFecSegMensal.cfm?mes={}&ano={}&regiao=SE&distribuidora={}&tipo=d'.format(mes,ano,distribuidora))
            dfs= pd.DataFrame(scraping[0])
            dfs.drop(dfs.tail(3).index,inplace=True)
            dfs.drop(dfs.head(3).index,inplace=True)
            dfs = dfs.assign(MES = '{}'.format(mes))
            dfs = dfs.assign(ANO = '{}'.format(ano))
            dfs = dfs.assign(DISTRIBUIDORA = '{}'.format(distribuidora))
            all_dfs = pd.DataFrame(dfs)
            all_dfs.to_csv('final_data_1.csv', encoding= 'utf-8', mode = 'a', header = False, index=False)            
            

In [ ]:
df = pd.read_csv("data.csv", encoding = 'utf-8',
                  names=['CONJUNTO', 'CÓDIGO', 'N_CONSUMIDORES','DECTOT','FECTOT','DECXP','FECXP','DECXN','FECXN','DECIP','FECIP','DECIND','FECIND','DECINE','FECINE','DECINC','FECINC','DECINO','FECINO','DECIPC','FECIPC','DECXPC','FECXPC','DECXNC','FECXNC','MES','ANO','COD_DISTRIBUIDORA'])

In [ ]:
df['DECTOT'] = df['DECTOT'].astype(int) / 100
df['FECTOT'] = df['FECTOT'].astype(int) /100
df['DECXP'] = df['DECXP'].astype(int) /100
df['FECXP'] = df['FECXP'].astype(int) /100
df['DECXN'] = df['DECXN'].astype(int) /100
df['FECXN'] = df['FECXN'].astype(int) /100
df['DECIP'] = df['DECIP'].astype(int) /100
df['FECIP'] = df['FECIP'].astype(int) /100
df['DECIND'] = df['DECIND'].astype(int) /100
df['FECIND'] = df['FECIND'].astype(int) /100
df['DECINE'] = df['DECINE'].astype(int) /100
df['FECINE'] = df['FECINE'].astype(int) /100
df['DECINC'] = df['DECINC'].astype(int) /100
df['FECINC'] = df['FECINC'].astype(int) /100
df['FECINO'] = df['FECINO'].astype(int) /100
df['DECIPC'] = df['DECIPC'].astype(int) /100
df['FECIPC'] = df['FECIPC'].astype(int) /100
df['DECXPC'] = df['DECXPC'].astype(int) /100
df['FECXPC'] = df['FECXPC'].astype(int) /100
df['DECXNC'] = df['DECXNC'].astype(int) /100
df['FECXNC'] = df['FECXNC'].astype(int) /100
df['N_CONSUMIDORES'] = df['N_CONSUMIDORES'].astype(float,0)

In [ ]:
df['DEC_APURADO'] = df['DECXP'] + df['DECXN'] + df['DECIP'] + df['DECIND']
df['FEC_APURADO'] = df['FECXP'] + df['FECXN'] + df['FECIP'] + df['FECIND']

In [ ]:
cols_to_order = ['CONJUNTO', 'CÓDIGO', 'N_CONSUMIDORES', 'DEC_APURADO','FEC_APURADO']
new_columns = cols_to_order + (df.columns.drop(cols_to_order).tolist())
df = df[new_columns]

In [ ]:
conn = sqlite3.connect('qualidade_energia')
c = conn.cursor()

In [ ]:
c.execute('CREATE TABLE IF NOT EXISTS dec_fec (CONJUNTO text, CÓDIGO number,CONSUMIDORES number, DECTOT number,FECTOT number,DECXP number,FECXP number,DECXN number,FECXN number,DECIP number,FECIP number,DECIND number,FECIND number,DECINE number,FECINE number,DECINC number,FECINC number, DECINO number,FECINO number,DECIPC number,FECIPC number,DECXPC number,FECXPC number, DECXNC number,FECXNC number,MES text,ANO text,COD_DISTRIBUIDORA text)')
conn.commit()

In [ ]:
df.to_sql('dec_fec', conn, if_exists='replace', index = False)